# Register Datasets

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
keypoint_names = ["front"]
keypoint_flip_map = []

register_coco_instances("50img_train", {}, r"/home/main/Desktop/detectron2/datasets/50img_coco/annotations/train.json", r"/home/main/Desktop/detectron2/datasets/50img_coco/images")
register_coco_instances("50img_val", {}, r"/home/main/Desktop/detectron2/datasets/50img_coco/annotations/val.json", r"/home/main/Desktop/detectron2/datasets/50img_coco/images/")
MetadataCatalog.get("50img_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("50img_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

register_coco_instances("merged_train", {}, r"/home/main/Desktop/detectron2/datasets/merged/annotations/train.json", r"/home/main/Desktop/detectron2/datasets/merged/images")
register_coco_instances("merged_val", {}, r"/home/main/Desktop/detectron2/datasets/merged/annotations/val.json", r"/home/main/Desktop/detectron2/datasets/merged/images/")
MetadataCatalog.get("merged_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("merged_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

register_coco_instances("merged_oc_train", {}, r"/home/main/Desktop/detectron2/datasets/merged_oc/annotations/train.json", r"/home/main/Desktop/detectron2/datasets/merged_oc/images")
register_coco_instances("merged_oc_val", {}, r"/home/main/Desktop/detectron2/datasets/merged_oc/annotations/val.json", r"/home/main/Desktop/detectron2/datasets/merged_oc/images/")
MetadataCatalog.get("merged_oc_train").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)
MetadataCatalog.get("merged_oc_val").set(keypoint_names=keypoint_names, keypoint_flip_map=keypoint_flip_map)

evaluation_dataset_path = r"/home/main/Desktop/detectron2/datasets/testing"

ModuleNotFoundError: ignored

# Mask R-CNN

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer,DefaultPredictor
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.visualizer import Visualizer
from detectron2.data.transforms import RandomFlip, RandomCrop, RandomRotation, RandomBrightness, RandomContrast
from detectron2.data import DatasetMapper, build_detection_train_loader
import torch
import cv2
import os

output_dir = r"/home/main/Desktop/detectron2/manipulator/evaluation"
val_dataset_path = r"/home/main/Desktop/detectron2/datasets/testing/"

dataset = "merged"

class CustomTrainerAugment(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            if not os.path.exists(os.path.join(cfg.OUTPUT_DIR, "eval")):
                os.mkdir(os.path.join(cfg.OUTPUT_DIR, "eval"))
            output_folder = os.path.join(cfg.OUTPUT_DIR, "eval")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(
            cfg,
            is_train=True,
            augmentations=[
                RandomBrightness(0.5, 2),
                RandomContrast(0.5, 2),
                RandomFlip(prob=0.5, horizontal=True),  # Flip the images horizontally and vertically with a probability of 0.5
                RandomFlip(prob=0.5, horizontal=False, vertical=True)
                #RandomCrop("absolute", (400, 400)),  # Crop the images to a size of 400x400 pixels
                #RandomRotation(angle=[90, 20], expand=True)  # Rotate the images 90 degrees
            ]
        )
        return build_detection_train_loader(cfg, mapper=mapper)
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            if not os.path.exists(os.path.join(cfg.OUTPUT_DIR, "eval")):
                os.mkdir(os.path.join(cfg.OUTPUT_DIR, "eval"))
            output_folder = os.path.join(cfg.OUTPUT_DIR, "eval")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

def train(cfg):
    if "oc" in cfg.DATASETS.TRAIN:
        trainer = CustomTrainerAugment(cfg)
    else:
        trainer = CustomTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    return trainer

def save(cfg, trainer):
    torch.save(trainer.model, os.path.join(cfg.OUTPUT_DIR, 'checkpoint.pth'))
    torch.save(trainer.model.state_dict(), os.path.join(cfg.OUTPUT_DIR, "checkpoint_state_dict.pth"))
def eval(cfg):
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "checkpoint_state_dict.pth")
    predictor = DefaultPredictor(cfg)
    for root, dirs, files in os.walk(val_dataset_path):
      folder = root.split('/')[-1]
      if not os.path.exists(os.path.join(evaluation_dataset_path, folder)):
          os.makedirs(os.path.join(evaluation_dataset_path, folder))
      for image in files:
          print(image)
          img = cv2.imread(os.path.join(root, image))
          outputs = predictor(img)
          v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
          v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
          out = v.get_image()[:, :, ::-1]
          cv2.imwrite(os.path.join(evaluation_dataset_path, folder, image), out)

## R-50 Backbone

### 1x LR

#### One Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_oc_train",)
cfg.DATASETS.TEST = (dataset+"_oc_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_50_FPN_1x.pkl"
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r50-1x-lr%s-it%s-oc" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

# with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
#     yaml.write(cfg.dump())

#save(cfg, trainer)
#eval(cfg)

In [ ]:
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 400
trainer = CustomTrainerAugment(cfg)
trainer.resume_or_load(resume=False)
for param in trainer.model.backbone.parameters():
    param.requires_grad = False
trainer.train()
trainer = train(cfg)

In [ ]:
cfg.SOLVER.WARMUP_ITERS = 50
cfg.SOLVER.MAX_ITER = 600
trainer = CustomTrainerAugment(cfg)
for param in trainer.model.backbone.parameters():
    param.requires_grad = True
trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
cfg.SOLVER.WARMUP_ITERS = 50
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 700
trainer = CustomTrainerAugment(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

#### Multi Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_train",)
cfg.DATASETS.TEST = (dataset+"_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_50_FPN_1x.pkl"
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 2000
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r50-1x-lr%s-it%s" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
    yaml.write(cfg.dump())

trainer = train(cfg)
save(cfg, trainer)
eval(cfg)

### 3x LR

#### One Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_oc_train",)
cfg.DATASETS.TEST = (dataset+"_oc_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_50_FPN_3x.pkl"
cfg.SOLVER.WARMUP_ITERS = 150
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r50-3x-lr%s-it%s-oc" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = 50
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

#with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
#    yaml.write(cfg.dump())

# trainer = train(cfg)
# save(cfg, trainer)
# eval(cfg)

In [ ]:
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = (800,)
trainer = CustomTrainerAugment(cfg)
trainer.resume_or_load(resume=False)
for param in trainer.model.backbone.parameters():
    param.requires_grad = False
trainer.train()

NameError: ignored

In [ ]:
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1200
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = (2200, 2700)
trainer = CustomTrainerAugment(cfg)
for param in trainer.model.backbone.parameters():
    param.requires_grad = True
trainer.resume_or_load(resume=True)
trainer.train()

[07/04 14:09:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/home/main/miniconda3/lib/python3.10/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays

[07/04 14:09:54 d2.utils.events]:  eta: 0:45:31  iter: 1019  total_loss: 4.704  loss_cls: 0.1542  loss_box_reg: 0.2483  loss_mask: 0.2491  loss_keypoint: 3.843  loss_rpn_cls: 0.04965  loss_rpn_loc: 0.1018    time: 1.3834  last_time: 1.3769  data_time: 0.0338  last_data_time: 0.0134   lr: 0.0016986  max_mem: 6642M
[07/04 14:10:22 d2.utils.events]:  eta: 0:45:12  iter: 1039  total_loss: 4.675  loss_cls: 0.1485  loss_box_reg: 0.2403  loss_mask: 0.2221  loss_keypoint: 3.948  loss_rpn_cls: 0.04088  loss_rpn_loc: 0.09622    time: 1.3864  last_time: 1.3709  data_time: 0.0225  last_data_time: 0.0133   lr: 0.0017319  max_mem: 6642M
[07/04 14:10:36 d2.data.datasets.coco]: Loaded 75 images in COCO format from /home/main/Desktop/detectron2/datasets/merged_oc/annotations/val.json
[07/04 14:10:36 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    tip1    | 385          |
|            |              |
[07/04 14:10:36 d2.d

#### Multi Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_oc_train",)
cfg.DATASETS.TEST = (dataset+"_oc_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_50_FPN_3x.pkl"
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (int(cfg.SOLVER.MAX_ITER*.65), int(cfg.SOLVER.MAX_ITER*0.85))
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r50-3x-lr%s-it%s-oc" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
    yaml.write(cfg.dump())

trainer = train(cfg)
save(cfg, trainer)
eval(cfg)

## R101 Backbone

### 3x LR

#### One Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_oc_train",)
cfg.DATASETS.TEST = (dataset+"_oc_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_101_FPN_3x.pkl"
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.STEPS = (int(cfg.SOLVER.MAX_ITER*.65), int(cfg.SOLVER.MAX_ITER*0.85))
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r101-3x-lr%s-it%s-oc" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.MASK_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = 50
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
    yaml.write(cfg.dump())

In [ ]:
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = (800,)
trainer = CustomTrainerAugment(cfg)
trainer.resume_or_load(resume=False)
for param in trainer.model.backbone.parameters():
    param.requires_grad = False
trainer.train()

In [ ]:
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1200
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = (2200, 2700)
trainer = CustomTrainerAugment(cfg)
for param in trainer.model.backbone.parameters():
    param.requires_grad = True
trainer.resume_or_load(resume=True)
trainer.train()

#### Multi Class

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset+"_oc_train",)
cfg.DATASETS.TEST = (dataset+"_oc_val",)
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = r"/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_101_FPN_3x.pkl"
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (int(cfg.SOLVER.MAX_ITER*.65), int(cfg.SOLVER.MAX_ITER*0.85))
cfg.OUTPUT_DIR = os.path.join(output_dir, "maskeyrcnn-r101-3x-lr%s-it%s-oc" % (cfg.SOLVER.BASE_LR, cfg.SOLVER.MAX_ITER))
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_INSTANCES_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE = 0

cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER // 20
cfg.TEST.KEYPOINT_OKS_SIGMAS = (1,)
cfg.TEST.DETECTIONS_PER_IMAGE = 8
cfg.TEST.AUG.ENABLED = True

with open(cfg.OUTPUT_DIR + "/config.yaml", 'w') as yaml:
    yaml.write(cfg.dump())

trainer = train(cfg)
save(cfg, trainer)
eval(cfg)

# Combine Checkpoints

In [ ]:
import pickle
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# Load the keypoint head using the first checkpoint file (pickle format)
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model)
with open(r"/home/main/Desktop/detectron2/manipulator/checkpoints/COCO-Keypoints-keypoint_rcnn_R_101_FPN_3x-138363331-model_final_997cc7.pkl", "rb") as f:
    first_checkpoint = pickle.load(f)
#print(first_checkpoint['model'].keys())


# Load the mask head parameters from the second checkpoint file (pickle format)
with open(r"/home/main/Desktop/detectron2/manipulator/checkpoints/COCO-InstanceSegmentation-mask_rcnn_R_101_FPN_3x-138205316-model_final_a3ec72.pkl", "rb") as f:
    second_checkpoint = pickle.load(f)

mask_head_parameters = {k: v for k, v in second_checkpoint["model"].items() if "mask" in k}

print(mask_head_parameters.keys())

for param in mask_head_parameters.keys():
    first_checkpoint["model"][param] = mask_head_parameters[param]

# Save the combined model to a final checkpoint file (pickle format)
with open("/home/main/Desktop/detectron2/manipulator/checkpoints/maskey_rcnn_R_101_FPN_3x.pkl", "wb") as f:
    pickle.dump(first_checkpoint, f)